In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
#Baseline
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif

# For matrics
from scipy.sparse import csr_matrix

# Factorization
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from catboost import CatBoostClassifier, Pool
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

In [4]:
def run_grid_search(estimator, X, params_grid, cv, scoring='roc_auc'):
    if 'target' in X:
        X = X.sample(frac=0.3, replace=True, random_state=RANDOM_STATE).reset_index(drop=True)
        y = X['target']
        X = X.drop('target', axis=1)
    gsc = GridSearchCV(estimator, params_grid, scoring=scoring, cv=cv, n_jobs=-1)

    gsc.fit(X, y)
    print("Best %s score: %.2f" % (scoring, gsc.best_score_))
    print()
    print("Best parameters set found on development set:")
    print()
    print(gsc.best_params_)
    print()
    print("Grid scores on development set:")
    print()

    for i, params in enumerate(gsc.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (gsc.cv_results_['mean_test_score'][i], gsc.cv_results_['std_test_score'][i] * 2, params))

    print()
    
    return gsc

# 1. Download Data

In [5]:
train = pd.read_csv('data_train.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('data_test.csv').drop('Unnamed: 0', axis=1)

In [6]:
feat = pd.read_csv('features.csv', sep='\t', header=0).drop('Unnamed: 0', axis=1)

# 2. Columns

In [7]:
USER_COL = 'id'
SERVICE_COL = 'vas_id'
ACTUAL_COL = 'bought_service'

# 3. Important var

In [8]:
common_users = train[USER_COL].unique()
RANDOM_STATE = 21
train_users = train.id.tolist()
test_users = test.id.tolist()

# 4. Data discover

# 4.1 % of "cold" users

In [9]:
# labels = ['common_users', 'solo_test_user']
# colors = ['yellow','green']
# plt.title('Cold users')
# plt.pie([train[USER_COL].nunique(), test[~test[USER_COL].isin(common_users)][USER_COL].nunique()],
#         labels=labels,
#         colors=colors,
#         startangle=180,
#         autopct = "%0.2f%%",)
# plt.figure(figsize=(8, 6), dpi=80)
# plt.show()

# 4.2 Target Balance

In [10]:
# plt.title('Our service reccomend - purchase history')
# plt.bar(height=train.target.value_counts(), x=["Didn't buy", 'buy'])

## 4.3 How much services did users from train buy?

In [11]:
# train_user_service_df = train.groupby(USER_COL)[SERVICE_COL].unique().reset_index()
# train_user_service_df.columns=[USER_COL, ACTUAL_COL]
# train_user_service_df['bought_services'] = train_user_service_df[ACTUAL_COL].apply(lambda x: len(x))
# train_user_service_df['bought_services'].value_counts()

In [12]:
# plt.title('How much services do user have?')
# plt.bar(height=train_user_service_df['bought_services'].value_counts(), x=["1", '2', '3'])

## 4.4 What service user prefer ?

In [13]:
# plt.title('Which services are popular?')
# plt.bar(height=train[SERVICE_COL].value_counts(), x=[i + 1 for i in range(len(train[SERVICE_COL].value_counts()))])


# 5. Feature Engineering

In [ ]:
X_1 = train.merge(feat, on='id', how='left')
X_2 = test.merge(feat, on='id', how='left')

In [ ]:
X_1 = X_1.drop(['buy_time_x', 'buy_time_y'], axis=1)
X_2 = X_2.drop(['buy_time_x', 'buy_time_y'], axis=1)

In [ ]:
n_unique_v_of_feature = X_1.nunique()

In [ ]:
# n_unique_v_of_feature.hist(bins=10)

#### all features

In [ ]:
f_all = set(n_unique_v_of_feature.index.tolist())
len(f_all)

#### constant features

In [ ]:
f_const = set(n_unique_v_of_feature[n_unique_v_of_feature == 1].index.tolist())
len(f_const)

#### numeric features

In [ ]:
f_numeric = set(n_unique_v_of_feature[n_unique_v_of_feature >= 10].index.tolist())
len(f_numeric)

#### cat features

In [ ]:
f_cat = set(n_unique_v_of_feature[n_unique_v_of_feature < 10].index.tolist())
len(f_cat)

In [ ]:
f_other = f_all - (f_const | f_numeric | f_cat)
len(f_other)

# 6. Make Pipeline

In [ ]:
class PreProcessing(BaseEstimator, TransformerMixin):
    """Custom Pre-Processing estimator for our use-case
    """

    def __init__(self):
        pass

    def transform(self, df):
        """Regular transform() that is a help for training, validation & testing datasets
           (NOTE: The operations performed here are the ones that we did prior to this cell)
        """
        try:
            df = df.drop('vas_id', axis=1)
        except:
            print("Dosn't have vas_id")
        
        # Categorical data
        for cat in f_cat:
            if cat not in ['target', 'vas_id']:
                 df.loc[:, (cat)] =  df.loc[:, (cat)].fillna(df[cat].mode()[0])
            
        # Numeric data
        scaler = StandardScaler()
        for col in f_numeric:
            df.loc[:, (col)] =  df.loc[:, (col)].fillna(df[col].mode()[0])
            df.loc[:, (col)] = scaler.fit_transform(np.array(df.loc[:, (col)]).reshape(-1, 1))

        
        return df

    def fit(self, df, y=None, **fit_params):
        """Fitting the Training dataset & calculating the required values from train
           e.g: We will need the mean of X_train['Loan_Amount_Term'] that will be used in
                transformation of X_test
        """
        
        return self 

In [ ]:
# cat_pipe = Pipeline([
#     ('selector', ColumnSelector(f_cat)),
#     ('imputer', SimpleImputer(strategy="most_frequent")), 
# ])

# num_pipe = Pipeline([
#     ('selector', ColumnSelector(f_numeric)),
#     ('imputer', SimpleImputer(strategy="mean")),
#     ('scaler', StandardScaler())
# ])

# preprocessor = FeatureUnion([
#     ('cat', cat_pipe),
#     ('num', num_pipe)
# ])

# pipe = Pipeline([
#     ('preprocess', preprocessor),
#     ('model', CatBoostClassifier())
# ])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
pipe = make_pipeline(PreProcessing(),
                    GradientBoostingClassifier())

# 7. Validation

In [ ]:
# train_idx, test_idx = train_test_split(X.index, test_size=0.25, random_state=RANDOM_STATE)

# y_train, y_test = y.loc[train_idx], y.loc[test_idx]
# X_train, X_test = X.loc[train_idx], X.loc[test_idx]

# print("Train")
# print(y.loc[train_idx].value_counts()/train_idx.shape[0])
# print()
# print("Test")
# print(y.loc[test_idx].value_counts()/test_idx.shape[0])

In [ ]:
param_grid = {"gradientboostingclassifier__n_estimators" : [10, 20, 30],
             "gradientboostingclassifier__max_depth" : [None, 6, 8, 10],
             "gradientboostingclassifier__max_leaf_nodes": [None, 5, 10, 20], 
             "gradientboostingclassifier__min_impurity_decrease": [0.1, 0.2, 0.3]}

gsc = run_grid_search(pipe, X_1, param_grid, cv=5)

In [ ]:
len(X_1)